# AI Agent:

Agent - Agents use an LLM to determine which actions to take and in what order. An action can either be using a tool and observing its output, or returning to the user.

## Agent Otaku:

Agent Otaku is an simple agent that has information about all the anime in the world, we can ask exciting questions to research and get and overview or understanding about Anime which can help us take real life decisions like should I binge this anime or the plot isn't exciting enough. We can also search based on characters!

In [ ]:
#importing necessary libraries
!pip install openai==0.28
!pip install httpx
import openai
import re
import httpx

In [ ]:
#ApiKey of the LLM
openai.api_key = "Masked API"

In [ ]:
#Our Core Otaku Agent Class
class Otaku:
    #Constructor
    def __init__(self, system=""):
        self.system = system
        self.messages = []
        if self.system:
            self.messages.append({"role": "system", "content": system})

    #Generating the response
    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    #Interacting with Chat Gpt 3.5 with this
    def execute(self):
        completion = openai.ChatCompletion.create(model="gpt-3.5-turbo", messages=self.messages)
        return completion.choices[0].message.content

In [ ]:
#Prompt for the LLM: taken some reference from professors prompt
prompt = """
You run stepwise as Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.
Your available actions are:
Anime:
e.g. Anime: Solo Leveling
Returns a summary and recent news about the term searching from MyAnimeList
Example session:
Question: Recent Updates on Solo Leveling
Thought: I should look up Solo Leveling on Kitsu
Action: Anime: Solo Leveling
PAUSE
You will be called again with this:
Observation: Solo Leveling is game based Anime where Sung Jin Woo the Player follows a system to level up to become the Strongest.
You then output:
Answer: Sung Jin Woo he player becomes strongest becoming player and follwing the System.
""".strip()

In [ ]:
#Function to retrive data from Kitsu.io
def myanimelist(query):
    # Note: searching by title.
    response = httpx.get(f"https://kitsu.io/api/edge/anime", params={
        "filter[text]": query
    })

    # Parseing the JSON response.
    data = response.json()

    # Checking if there is any anime returned.
    if not data or "data" not in data or not data["data"]:
        return "No results found."

    # Extracting the first anime's attributes for simplicity.
    anime = data["data"][0]
    title = anime["attributes"]["canonicalTitle"]
    synopsis = anime["attributes"]["synopsis"]

    # Return a formatted string with the title and synopsis.
    return f"Title: {title}\nSynopsis: {synopsis}"

#Available Actions, Mapping Action to function
known_actions = {
    "Anime": myanimelist
}

In [ ]:
#Parsing through the query using regular expressions
action_re = re.compile('^Action: (\w+): (.*)$')
def query(question, max_turns=2):
    i = 0
    #Initializing the Agent
    bot = Otaku(prompt)
    #asking the question
    next_prompt = question
    while i < max_turns:
        i += 1
        #recording the thought process and printing it
        result = bot(next_prompt)
        print(result)
        #retriving the action
        actions = [action_re.match(a) for a in result.split('\n') if action_re.match(a)]
        #checking if there is a match and thus an action to run
        if actions:
            # There is an action to run
            action, action_input = actions[0].groups()
            #Checking if action is a known action
            if action not in known_actions:
                raise Exception("Unknown action: {}: {}".format(action, action_input))
            print(" -- running {} {}".format(action, action_input))
            observation = known_actions[action](action_input)
            print("Observation:", observation)
            next_prompt = "Observation: {}".format(observation)
        else:
            return None

Asking our Otaku about the legendary Saiyan Goku :)

In [ ]:
#Asking the question to Otaku
query('Who is Goku?')

Thought: Goku is a widely recognized character from the Dragon Ball series.

Action: Anime: Dragon Ball
PAUSE
 -- running Anime Dragon Ball
Observation: Title: Dragon Ball
Synopsis: Gokuu Son is a young boy who lives in the woods all alone—that is, until a girl named Bulma runs into him in her search for a set of magical objects called the "Dragon Balls." Since the artifacts are said to grant one wish to whoever collects all seven, Bulma hopes to gather them and wish for a perfect boyfriend. Gokuu happens to be in possession of a dragon ball, but unfortunately for Bulma, he refuses to part ways with it, so she makes him a deal: he can tag along on her journey if he lets her borrow the dragon ball's power. With that, the two set off on the journey of a lifetime.
They don't go on the journey alone. On the way, they meet the old Muten-Roshi and wannabe disciple Kuririn, with whom Gokuu trains to become a stronger martial artist for the upcoming World Martial Arts Tournament. However, it's

Each part of the process is well commented and refrences have been mentioned.